# Exploring MatGL: Machine Learning Interatomic Potentials

**MatGL** is a Python framework for materials graph learning that provides fast and accurate interatomic potentials trained on large databases like the Materials Project. This 10-minute tutorial will show you how to:

1. Create crystal structures using pymatgen
2. Load pre-trained machine learning potentials (M3GNet)
3. Relax structures to find equilibrium geometries
4. Visualize results in 3D

**Why MatGL?** Traditional molecular dynamics uses expensive DFT calculations or empirical force fields. MatGL offers DFT-level accuracy at a fraction of the computational cost using graph neural networks.

## Step 1: Import Libraries

We'll use:
- `matgl`: The main framework for graph-based ML potentials
- `pymatgen`: For creating and manipulating crystal structures
- `ase`: Atomic Simulation Environment for structure relaxation
- `py3Dmol`: For interactive 3D visualization

In [ ]:
import matgl
from matgl.ext.ase import Relaxer
from pymatgen.core import Structure, Lattice
import py3Dmol
import warnings
warnings.filterwarnings('ignore')

# Set backend to PyTorch Geometric (we use PYG instead of DGL on macOS)
matgl.set_backend("PYG")
print("MatGL backend set to: PYG")

## Step 2: Create a Crystal Structure

Let's create a **body-centered cubic (BCC) Molybdenum** structure. We'll intentionally start with a distorted lattice (3.3 Å) that's larger than the DFT-optimized value (~3.16 Å) to demonstrate the relaxation capabilities.

In BCC, atoms are at positions (0,0,0) and (0.5,0.5,0.5) in fractional coordinates.

In [ ]:
# Create a "distorted" Molybdenum BCC structure
# (The true DFT lattice constant is ~3.16 Å; we'll start at 3.3 Å)
coords = [[0, 0, 0], [0.5, 0.5, 0.5]]
lattice = Lattice.cubic(3.3) 
mo_structure = Structure(lattice, ["Mo", "Mo"], coords)

print(f"Initial structure:")
print(f"  Formula: {mo_structure.composition}")
print(f"  Lattice parameter: {mo_structure.lattice.a:.3f} Å")
print(f"  Number of atoms: {len(mo_structure)}")

## Step 3: Load the M3GNet Universal Potential

**M3GNet** (Materials 3-body Graph Network) is a universal potential trained on the Materials Project database. It can predict energies, forces, and stresses for virtually any inorganic crystal structure.

The model we're loading (M3GNet-MP-2021.2.8-PES) was trained on hundreds of thousands of DFT calculations.

In [ ]:
# Load the M3GNet Universal Potential
# This model was trained on the Materials Project (MP) database
print("Loading M3GNet model...")
model = matgl.load_model("M3GNet-MP-2021.2.8-PES")
print("Model loaded successfully!")

## Step 4: Relax the Structure

The **Relaxer** class uses the ML potential to optimize both:
- **Atomic positions**: Find the lowest energy arrangement of atoms
- **Lattice parameters**: Adjust the cell size and shape

This is similar to DFT geometry optimization but much faster!

In [ ]:
# Initialize the Relaxer
# This handles the optimization of both atom positions and lattice shape
relaxer = Relaxer(potential=model)

# Perform the relaxation
print("Starting M3GNet relaxation...")
results = relaxer.relax(mo_structure)
print("Relaxation complete!")

# Extract results
final_structure = results["final_structure"]
final_energy = results["trajectory"].energies[-1]

print(f"\nResults:")
print(f"  Initial Lattice: 3.300 Å")
print(f"  Relaxed Lattice: {final_structure.lattice.a:.3f} Å (M3GNet)")
print(f"  Final Energy:   {final_energy:.4f} eV")
print(f"  Lattice change: {((final_structure.lattice.a - 3.3) / 3.3 * 100):.1f}%")

## Step 5: Visualize the Structure in 3D

Let's use **3Dmol.js** to create an interactive 3D visualization. You can:
- Rotate: Click and drag
- Zoom: Scroll or pinch
- Pan: Right-click and drag

In [ ]:
def view_structure(structure, title="Structure"):
    """
    Create an interactive 3D view of a structure using 3Dmol.js
    """
    # Convert structure to XYZ format for visualization
    xyz = structure.to(fmt="xyz")
    
    # Create 3Dmol viewer
    viewer = py3Dmol.view(width=600, height=400)
    viewer.addModel(xyz, "xyz")
    
    # Style: spheres for atoms, sticks for bonds
    viewer.setStyle({'sphere': {'radius': 0.5}, 'stick': {'radius': 0.15}})
    
    # Add unit cell box
    viewer.addUnitCell()
    
    # Set view
    viewer.zoomTo()
    
    return viewer

print("Initial (distorted) structure:")
view_structure(mo_structure, "Initial Mo Structure")

In [ ]:
print("Relaxed structure:")
view_structure(final_structure, "Relaxed Mo Structure")

## What Just Happened?

1. **Created a distorted structure**: Started with Mo atoms in a cubic lattice (3.3 Å)
2. **Loaded ML potential**: Used a pre-trained M3GNet model trained on Materials Project data
3. **Relaxed the geometry**: The model predicted forces and optimized the structure
4. **Found equilibrium**: The lattice contracted to ~3.16 Å, close to the DFT value

**The power of MatGL**: This calculation took seconds, while a DFT calculation would take minutes to hours. The accuracy is comparable to DFT for many properties!

## Next Steps: Try It Yourself!

Here are some experiments to extend your learning:

1. **Different materials**: Try other BCC metals (Fe, W, Cr)
   ```python
   fe_structure = Structure(Lattice.cubic(2.9), ["Fe", "Fe"], coords)
   ```

2. **Different crystal structures**: Create FCC or HCP structures
   ```python
   # FCC example
   fcc_coords = [[0, 0, 0], [0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]]
   al_structure = Structure(Lattice.cubic(4.05), ["Al"]*4, fcc_coords)
   ```

3. **Analyze the trajectory**: Examine how energy changes during relaxation
   ```python
   energies = results["trajectory"].energies
   import matplotlib.pyplot as plt
   plt.plot(energies)
   plt.xlabel('Step')
   plt.ylabel('Energy (eV)')
   ```

4. **Load structures from databases**: Use pymatgen to fetch structures from Materials Project
   ```python
   # Requires API key
   from pymatgen.ext.matproj import MPRester
   ```

## Key Concepts Review

- **MatGL**: Framework for materials property prediction using graph neural networks
- **M3GNet**: A universal potential that works for most inorganic materials
- **Relaxation**: Finding the minimum energy configuration by optimizing positions and lattice
- **Speed vs. Accuracy**: ML potentials are ~1000x faster than DFT with similar accuracy

**Resources**:
- [MatGL Documentation](https://github.com/materialsvirtuallab/matgl)
- [M3GNet Paper](https://www.nature.com/articles/s43588-022-00349-3)
- [Pymatgen Documentation](https://pymatgen.org/)
- [Materials Project](https://materialsproject.org/)